# 05_04: Math and plotting with Pandas

In [ ]:
import math
import collections
import dataclasses
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as pp

In [ ]:
gapminder = pd.read_csv('gapminder.csv', dtype_backend='pyarrow', engine='pyarrow')

In [ ]:
gapminder.info()

In [ ]:
gapminder.describe()

In [ ]:
gapminder['log_gdp_per_day'] = np.log10(gapminder['gdp_per_capita'] / 365.25)

In [ ]:
gapminder.head()

In [ ]:
gapminder_by_year = gapminder.set_index('year').sort_index()

In [ ]:
gapminder_by_country = gapminder.set_index('country').sort_index()

In [ ]:
gapminder_by_year.loc[1960].plot.scatter('log_gdp_per_day', 'life_expectancy');

In [ ]:
axes = gapminder_by_year.loc[1960].plot.scatter('log_gdp_per_day', 'life_expectancy', label=1960)
gapminder_by_year.loc[2015].plot.scatter('log_gdp_per_day', 'life_expectancy', label=2015, color='C1', ax=axes);

In [ ]:
axes = gapminder_by_year.loc[1960].plot.scatter('log_gdp_per_day', 'age5_surviving', label=1960)
gapminder_by_year.loc[2015].plot.scatter('log_gdp_per_day', 'age5_surviving', label=2015, color='C1', ax=axes);

In [ ]:
gapminder_by_country.loc['Italy'].sort_index().plot('year', 'life_expectancy');

In [ ]:
axes = gapminder_by_country.loc['Italy'].sort_values('year').plot('year', 'life_expectancy', label='Italy')
gapminder_by_country.loc['China'].sort_values('year').plot('year', 'life_expectancy', label='China', ax=axes)
gapminder_by_country.loc['USA'].sort_values('year').plot('year', 'life_expectancy', label='USA', ax=axes)

pp.axis(xmin=1900, xmax=2023)
pp.ylabel('life expectancy')

In [ ]:
gapminder.babies_per_woman.mean()

In [ ]:
gapminder.groupby('year').babies_per_woman.mean()

In [ ]:
gapminder.groupby('year').babies_per_woman.mean().plot()
pp.ylabel('babies per woman')

gapminder.groupby('year').age5_surviving.mean().plot(secondary_y=True)
pp.ylabel('age 5 survival [%]');

In [ ]:
gapminder.pivot_table('babies_per_woman', 'year', 'region')

In [ ]:
gapminder.pivot_table('babies_per_woman', 'year', 'region').plot()
pp.title('babies per woman');

In [ ]:
gapminder.pivot_table('age5_surviving', 'year', 'region').plot()